# Unsupervised Classification Attempt

https://www.tutorialspoint.com/gensim/gensim_doc2vec_model.htm#:~:text=Doc2Vec%20model%2C%20as%20opposite%20to,the%20words%20in%20the%20sentence.
https://www.kaggle.com/code/currie32/predicting-similarity-tfidfvectorizer-doc2vec/notebook


In [1]:
# import gensim
# import gensim.downloader as api
# dataset = api.load("text8")
# data = [d for d in dataset]

In [2]:
# def tagged_document(list_of_list_of_words):
#    for i, list_of_words in enumerate(list_of_list_of_words):
#       yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
# data_for_training = list(tagged_document(data))

In [3]:
# print(data_for_training [:1])

In [4]:
# model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)

In [5]:
# model.build_vocab(data_for_training)

In [6]:
# model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)

In [7]:
# print(model.infer_vector(['violent', 'means', 'to', 'destroy', 'the','organization']))

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from gensim import utils
from gensim.models import Doc2Vec

In [9]:
# texts = ["This is the first document.", "This is the second document.", "And this is the third one.", "Is this the first document?"]

In [10]:
# vectorizer = TfidfVectorizer(stop_words='english')
# X = vectorizer.fit_transform(texts)

In [11]:
# kmeans = KMeans(n_clusters=2, random_state=0)
# kmeans.fit(X)

In [12]:
# print(kmeans.labels_)

In [13]:
bellingcat_df = pd.read_csv('../data/bellingcat.csv')
ceninfores = pd.read_csv('../data/ceninfores.csv')
get_geoconfirmed_data = pd.read_csv('../data/get_geoconfirmed_data.csv')
defmon = pd.read_csv('../data/defmon.csv')

In [14]:
bellingcat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1115 entries, 0 to 1114
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           1115 non-null   object 
 1   date         1115 non-null   object 
 2   latitude     1115 non-null   float64
 3   longitude    1115 non-null   float64
 4   place_desc   930 non-null    object 
 5   title        0 non-null      float64
 6   description  1115 non-null   object 
 7   source       1115 non-null   object 
 8   links        1115 non-null   object 
 9   filters      1115 non-null   object 
dtypes: float64(3), object(7)
memory usage: 87.2+ KB


In [15]:
ceninfores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10006 entries, 0 to 10005
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           10006 non-null  object 
 1   date         10006 non-null  object 
 2   latitude     10006 non-null  float64
 3   longitude    10006 non-null  float64
 4   place_desc   10006 non-null  object 
 5   title        0 non-null      float64
 6   description  10003 non-null  object 
 7   source       10006 non-null  object 
 8   links        10006 non-null  object 
 9   filters      10006 non-null  object 
dtypes: float64(3), object(7)
memory usage: 781.8+ KB


In [16]:
get_geoconfirmed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9047 entries, 0 to 9046
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   geoconfirmed_id  9047 non-null   object 
 1   icon             9047 non-null   object 
 2   id               9047 non-null   object 
 3   date             9047 non-null   object 
 4   latitude         9047 non-null   float64
 5   longitude        9047 non-null   float64
 6   place_desc       0 non-null      float64
 7   title            9047 non-null   object 
 8   description      9047 non-null   object 
 9   source           9047 non-null   object 
 10  links            9047 non-null   object 
 11  filters          9047 non-null   object 
dtypes: float64(3), object(9)
memory usage: 848.3+ KB


In [17]:
defmon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19272 entries, 0 to 19271
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           0 non-null      float64
 1   date         19272 non-null  object 
 2   latitude     19272 non-null  float64
 3   longitude    19272 non-null  float64
 4   place_desc   19170 non-null  object 
 5   title        19272 non-null  object 
 6   description  19255 non-null  object 
 7   source       19272 non-null  object 
 8   links        19272 non-null  object 
 9   filters      19272 non-null  object 
dtypes: float64(3), object(7)
memory usage: 1.5+ MB


In [18]:
def review_to_wordlist(review, remove_stopwords=True):
    # Clean the text, with the option to remove stopwords.
    
    # Convert words to lower case and split them
    words = review.lower().split()

    # Optionally remove stop words (true by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    review_text = " ".join(words)

    # Clean the text
    review_text = re.sub(r"[^A-Za-z0-9(),!.?\'\`]", " ", review_text)
    review_text = re.sub(r"\'s", " 's ", review_text)
    review_text = re.sub(r"\'ve", " 've ", review_text)
    review_text = re.sub(r"n\'t", " 't ", review_text)
    review_text = re.sub(r"\'re", " 're ", review_text)
    review_text = re.sub(r"\'d", " 'd ", review_text)
    review_text = re.sub(r"\'ll", " 'll ", review_text)
    review_text = re.sub(r",", " ", review_text)
    review_text = re.sub(r"\.", " ", review_text)
    review_text = re.sub(r"!", " ", review_text)
    review_text = re.sub(r"\(", " ( ", review_text)
    review_text = re.sub(r"\)", " ) ", review_text)
    review_text = re.sub(r"\?", " ", review_text)
    review_text = re.sub(r"\s{2,}", " ", review_text)
    
    words = review_text.split()
    
    # Shorten words to their stems
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in words]
    
    review_text = " ".join(stemmed_words)
    
    # Return a list of words
    return(review_text)

I need to pull out the title and description column for these data sources, and concatenate them into one massive file. I need to regex the words to get rid of artifact html text.

In [19]:
bellingcat_df['list_text'] = bellingcat_df['description']
ceninfores['list_text'] = ceninfores['description']
get_geoconfirmed_data['list_text'] = get_geoconfirmed_data['title'] + ' ' + get_geoconfirmed_data['description'] + get_geoconfirmed_data['icon']
defmon['list_text'] = defmon['title'] + ' ' + defmon['description']

In [20]:
#I need to drop the first 16 characters from each element in the geoconfirmed list. It is messing up the clustering I think
get_geoconfirmed_data['list_text'] = get_geoconfirmed_data['list_text'].str.slice(start=16)

In [21]:
#review text and create a dataframe of post features, just doing it for geoconfirmed right now
postfeatures = []

for i in get_geoconfirmed_data['list_text']:
    postfeatures.append(review_to_wordlist(i, remove_stopwords=True))

In [22]:
postfeatures

["uav tank russian southern militari district fight mar'inka donestk oblast background anoth use seem thermit br br geo br https twitter com geoconfirm status 1621920764008988672 04 1823 feb 1527 uav tank russian southern militari district fight mar'inka donestk oblast background anoth use seem thermit",
 "uav tank russian southern militari district fight mar'inka donestk oblast background anoth use seem thermit br br geo br https twitter com geoconfirm status 1621920764008988672 04 1823 feb 1527 uav tank russian southern militari district fight mar'inka donestk oblast background anoth use seem thermit",
 'uav russian infantri shell ukrainian forc br br geo br https twitter com geoconfirm status 1610664469587283974 04 0954 jan 2022 uav russian infantri shell ukrainian forc',
 'russian train logist vehicl mazyr belarus 3523 x98 boriskovichi belarus 15 1700 jan 2022',
 'russian train militari equip spot rechitsa tank armour personnel carrier various zil kamaz truck even excav seen platfo

In [23]:
#Weird but maybe useful :) vectorize them
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(postfeatures)

In [24]:
scores = []

for k in range(2, 12):
    cl = KMeans(n_clusters=k, random_state=42)
    cl.fit(X)
    inertia = cl.inertia_
    sil = silhouette_score(X, cl.labels_)
    
    scores.append([k, inertia, sil])
    
score_df = pd.DataFrame(scores)
score_df.columns = ['k', 'inertia', 'silhouette_score']

D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` w

In [25]:
score_df

,k,inertia,silhouette_score
0,2,8558.705772,0.019017
1,3,8496.012367,0.009162
2,4,8453.911273,0.006915
3,5,8419.107034,0.008281
4,6,8388.705349,0.006541
5,7,8376.477552,0.005322
6,8,8356.680679,0.005195
7,9,8339.838939,0.004463
8,10,8317.612649,0.004797
9,11,8307.075846,0.005233


In [26]:
km = KMeans(n_clusters=12, random_state=42)
km.fit(X)
get_geoconfirmed_data['clusters'] = km.predict(X)

D:\Anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [27]:
get_geoconfirmed_data.to_csv('../data/get_geoconfirmed_data_clusters.csv', index = False)

In [28]:
#The analyze the clusters